## First Neural Network: Image Classification 

Objectives:
- Train a minimal image classifier on [MNIST](https://paperswithcode.com/dataset/mnist) using PyTorch
- Usese PyTorch and torchvision

In [1]:
# 导入
import torch#安装torch库，支持在图形处理单元上计算张量
import torch.nn as nn#加载神经网络的常用模块
import torchvision#tochvision主要处理图像数据，包含一些常用的数据集、模型、转换函数等
import torchvision.transforms as transforms#提供了一系列的变换函数,可以通过组合这些函数来实现各种复杂的变换操作

In [2]:
# 加载数据
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size#赋初值

    def __call__(self, img):
        return torch.reshape(img, self.new_size)#改变img的大小为new_size

transformations = transforms.Compose([#图像预处理
                                transforms.ToTensor(),#把一个取值范围是[0,255]的PIL.Image或者shape为(H,W,C)的numpy.ndarray，转换成形状为[C,H,W]，取值范围是[0,1.0]的torch.FloadTensor
                                transforms.ConvertImageDtype(torch.float32),#输出的所需数据类型为浮点型32位
                                ReshapeTransform((-1,))#改变将输入Tensor的事件形状
                                ])

trainset = torchvision.datasets.MNIST(root='./data', train=True,#加载mnist数据集，从./data获取数据，train=True则为训练集
                                        download=True, transform=transformations)#对图像预处理

testset = torchvision.datasets.MNIST(root='./data', train=False,#加载mnist数据集，train=False则为测试集
                                       download=True, transform=transformations)#对图像预处理

In [3]:
# 检查数据shape
trainset.data.shape, testset.data.shape#查看数据大小

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [4]:
# 数据加载器
BATCH_SIZE = 128#一次参数学习所使用的样本数量为128
train_dataloader = torch.utils.data.DataLoader(trainset, #包装训练集数据
                                               batch_size=BATCH_SIZE,#批训练样本大小
                                               shuffle=True, #每次迭代训练时将数据洗牌
                                               num_workers=0)#使用num_workers个子进程来导入数据，0时使用主进程来导入数据

test_dataloader = torch.utils.data.DataLoader(testset, #包装测试集数据
                                              batch_size=BATCH_SIZE,#批训练样本大小
                                              shuffle=False, #每次迭代训练时不需要将数据洗牌
                                              num_workers=0)#使用num_workers个子进程来导入数据，0时使用主进程来导入数据

In [5]:
# 模型
model = nn.Sequential(nn.Linear(784, 512), nn.ReLU(), nn.Linear(512, 10))#进行网络构建
#我们所要构造的MLP是一个两层的网络，其中输入784个元素，隐藏层有512个单元，输出为10个标签，ReLU()使用激活函数

In [6]:
# 训练准备
trainer = torch.optim.RMSprop(model.parameters())#定义优化器
loss = nn.CrossEntropyLoss()#交叉熵损失函数作为损失函数来优化模型，通过度量两个概率分布的差异性的，来衡量模型学习到的分布和真实分布的差异

In [7]:
def get_accuracy(output, target, batch_size):#获得训练轮次的准确性
    corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).sum()#通过公式，计算训练正确个数
    accuracy = 100.0 * corrects/batch_size#准确率为正确个数/样本大小*100%
    return accuracy.item()#返回准确率取值

In [8]:
# 训练
for ITER in range(5):
    train_acc = 0.0#训练精度
    train_running_loss = 0.0#训练损失

    model.train()#模型训练
    for i, (X, y) in enumerate(train_dataloader):#批处理训练数据
        output = model(X)
        l = loss(output, y)#损失

        # 更新参数
        l.backward()#反向传播
        trainer.step()#以求得的导数和，结合优化器，更新参数，然后进行下一批次的训练
        trainer.zero_grad()#梯度清零

        # 收集度量
        train_acc += get_accuracy(output, y, BATCH_SIZE)#训练准确性加到训练精度上
        train_running_loss += l.detach().item()#不需要梯度回传的部分加到训练损失上

    print('Epoch: %d | Train loss: %.4f | Train Accuracy: %.4f' \
          %(ITER+1, train_running_loss / (i+1),train_acc/(i+1)))#打印每轮次训练的损失和精确度

Epoch: 1 | Train loss: 0.9943 | Train Accuracy: 91.7344
Epoch: 2 | Train loss: 0.1334 | Train Accuracy: 95.9422
Epoch: 3 | Train loss: 0.1030 | Train Accuracy: 96.8767
Epoch: 4 | Train loss: 0.0845 | Train Accuracy: 97.4997
Epoch: 5 | Train loss: 0.0735 | Train Accuracy: 97.8811


### Other things to try

- Evaluate on test set
- Plot loss curve
- Add more layers to the model